This is a notebook for processing TopN results for Confidence-aware Models CUBCF, CIBCF

In [ ]:
import sys # set path of locally installed lenskit_confidence module
sys.path.insert(0,'C:\\Users\\Name\\Documents\\GitHub\\lenskit_confidence')

In [ ]:
import pandas as pd
from lenskit import topn, util

In [ ]:
util.log_to_notebook() # let notebook print updates from functions

In [ ]:
data = 'ml10m' # Specify dataset to process - 'ml1m', 'ml10m', 'ml20m', 'jester', etc... 

In [ ]:
# Specify which model results are being analyzed
#model = 'CIBCF'
model = 'CUBCF_500'

In [ ]:
path = 'D:\\results_conf_aware_nbhd_KDD2021\\' + data + '\\confidence_aware\\' + model + '\\'  # where are results stored?

In [ ]:
list_lengths = list((1,2,3,4,5,6,7,8,9,10)) # what list size are we looking at? 
# For example, if you produce 10 recommendations per user, we want to look at metrics@k for k in {1,...,10}
new_r0 = 4 # CUBCF and CIBCF scripts have default r0 = 4.5; this is for "undoing" that so we can analyze different r0 values
num_jobs = 4 # how many workers to set up

In [ ]:
# Output information from the models runs
# Should be in the "path" directory specified above
runs = pd.read_csv(path + 'runs.csv')
runs.set_index('RunId', inplace = True)
truth = pd.read_csv(path + 'truth.csv')
truth = truth[['user', 'item', 'rating']]

In [ ]:
for list_length in list_lengths: # a simple loop to compute TopN results for each list length
    print(list_length)
    recs = pd.read_parquet(path + 'recommendations.parquet')
    recs['prediction'] = (recs['prediction']*recs['var']) + 4.5
    recs['score'] = (recs['prediction'] - new_r0)/recs['var']
    recs['rank'] = recs.groupby(['user', 'num_nbhr'])['score'].rank("dense", ascending = False)
    #recs['score'] = recs['prediction']
    recs = recs[['item', 'score', 'user','rank','RunId']]
    recs = recs[recs['rank'] <= list_length]
    rla = topn.RecListAnalysis(n_jobs = num_jobs)
    rla.add_metric(topn.ndcg) # precision, recall, recip_rank, dcg, ndcg
    rla.add_metric(topn.precision)
    topn_compute = rla.compute(recs, truth)
    topn_results = topn_compute.join(runs[['name', 'nnbrs']], on = 'RunId')
    #ndcg.fillna(0).groupby(['name', 'nnbrs'])['ndcg','precision'].mean()
    topn_results.fillna(0).groupby(['name', 'nnbrs'])['ndcg','precision'].mean().to_csv(path + 'results' + str(eval('list_length')) + '.csv')